In [1]:
import os
import time
import sys

In [2]:
import numpy as np
import sklearn
import sklearn.feature_extraction.text
import sklearn.multiclass
import sklearn.linear_model

In [3]:
from nltk import word_tokenize

In [4]:
from nltk.stem import WordNetLemmatizer

In [5]:
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

vectorizer = sklearn.feature_extraction.text.CountVectorizer(tokenizer=LemmaTokenizer(), binary=True)

In [6]:
import pandas as pd

In [7]:
import findspark
findspark.init("/usr/hdp/2.6.5.0-292/spark2")
import pyspark
sc = pyspark.SparkContext()

In [8]:
import pandas as pd
import pyspark
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [9]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("Word Count").getOrCreate()

In [10]:
data = sqlContext.read.csv('/user/maria_dev/data/newsarticles_article.csv', sep=',', escape='"', header=False, 
               inferSchema=True, multiLine=True)

In [11]:
df = data.rdd.map(lambda line: (line[0],line[5].split(" ")))

In [12]:
df =  spark.createDataFrame(df,["id", "words"])

In [13]:
from pyspark.ml.feature import CountVectorizer

In [14]:
cv = CountVectorizer(inputCol="words", outputCol="features", vocabSize=3, minDF=2.0)

In [16]:
model = cv.fit(df)

In [17]:
X = model.transform(df)

In [18]:
clf = sklearn.multiclass.OneVsRestClassifier(sklearn.linear_model.LogisticRegression())

In [19]:
tags = sqlContext.read.csv('/user/maria_dev/data/tags.csv', sep=',', header=True,inferSchema=True, multiLine=True)

In [20]:
Y = tags.rdd.map(lambda x:x['category_id']).toPandas()

In [23]:
X = X.toPandas()

In [26]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=1. / 3, random_state=1)

In [27]:
clf.fit(X_train, Y_train)

In [28]:
Y_hat = clf.predict_proba(X_test)

In [29]:
Y_hat = Y_hat >0.5

In [30]:
acc = np.sum(Y_hat == Y_test) / float(Y_hat.size)

In [32]:
acc

0.92667205
